In [ ]:
import tensorflow as tf
from flask import Flask, render_template, request, session, url_for, redirect, jsonify
import pymysql

#=========
import nltk
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import json
import pickle

from sklearn import preprocessing
import pandas as pd
import numpy as np
import scipy.stats
import scipy.spatial
from sklearn.model_selection import KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
import warnings
import sys
#from sklearn.utils.extmath import np.dot

#=========Database Connection===
#connection = pymysql.connect(host="localhost", user="root", password="root", database="chatbot")
#cursor = connection.cursor()
#============start=======chatbot============

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

with open("intents.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

#tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

#==============end====chatbot=================


app = Flask(__name__)
app.secret_key = 'random string'


@app.route('/index')
#@app.route('/')
def index():
    return render_template('index.html')


@app.route('/register', methods=["GET","POST"])
def register():
    if request.method == "POST":
        name = request.form.get("name")
        email = request.form.get("email")
        mobile = request.form.get("mobile")
        gender = request.form.get("gender")
        dob = request.form.get("dob")
        username = request.form.get("username")
        password = request.form.get("password")
        cursor.execute("insert into userdetails(fullname,gender,mobile,email,dob,username,password) values('"+name+"','"+gender+"','"+mobile+"','"+email+"','"+dob+"','"+username+"','"+password+"')")
        connection.commit()
        #return render_template('/index')
        return redirect(url_for('/index'))
    else:
        #return render_template('index.html')
        return redirect(url_for('/index'))


@app.route('/login', methods=["GET","POST"])
def login():
    msg = ''
    if request.method == "POST":
        session.pop('user',None)
        username = request.form.get("username")
        password = request.form.get("password")
        cursor.execute('SELECT * FROM userdetails WHERE username = %s AND password = %s', (username, password))
        account = cursor.fetchone()
        #print(account)
        if account:
            session['user'] = account[1]
            #return render_template('home.html')
            return redirect(url_for('chatbot'))
        else:
            # Account doesnt exist or username/password incorrect
            msg = 'Incorrect username/password!'
    #return render_template('index.html', msg=msg)
    return redirect(url_for('index'))


#logout code
@app.route('/logout')
def logout():
    session.pop('user')
    return redirect(url_for('index'))


@app.route('/home')
def home():
    if 'user' in session:
        return render_template('home.html', user=session['user'])
    return redirect(url_for('index'))

@app.route('/aboutus')
def aboutus():
    return render_template('aboutus.html')


@app.route('/contactus')
def contactus():
    return render_template('contactus.html')
#bot start======

@app.route('/')
def chatbot():

    return render_template('chatbot.html', user="abc")
valobt=0
allstringis=''
@app.route("/get")
def get_bot_response():
    global valobt,allstringis
    valobt+=1
    #valobtained=valobtained+1
    userText = request.args.get('msg')
    '''print(bag_of_words(userText, words))
    results = model11.predict([bag_of_words(userText, words)])
    print(results)
    results_index = numpy.argmax(results)
    print(results_index)
    print(results[0][results_index])
    oppred=results[0][results_index]
    outdatagot = " \n"
    print(oppred)'''


    results = model.predict([bag_of_words(userText, words)])
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']

    print(random.choice(responses))

    return str(' '+random.choice(responses))



def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)
#model11.load("model.tflearn")
#bot end======-------------


if __name__ == '__main__':
    #app.run(debug="True")
    app.run('0.0.0.0')


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from C:\Users\Rakshita\Desktop\portfolio-projects\chatbot\chatbot\model.tflearn
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.104:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Aug/2021 14:14:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2021 14:14:20] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:20] "GET /static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:20] "GET /static/css/blog.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:20] "GET /static/css/timeline.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:20] "GET /static/css/font-awesome.min.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:20] "GET /static/js/jquery-2.2.3.min.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:21] "GET /static/js/move-top.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:21] "GET /static/js/SmoothScroll.min.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:21] "GET /static/js/bootstrap.min.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Aug/2021 14:14:21] "GET /static/css/bot.css HTTP/1.1" 304 -
127

Hi there, how can I help?


127.0.0.1 - - [19/Aug/2021 14:14:32] "GET /get?msg=pharamacy HTTP/1.1" 200 -


Happy to help!


...

Please provide pharmacy name


127.0.0.1 - - [19/Aug/2021 14:15:04] "GET /get?msg=search+hospital+near+me HTTP/1.1" 200 -


fortis,sion hospitsl


127.0.0.1 - - [19/Aug/2021 14:17:48] "GET /get?msg=hi HTTP/1.1" 200 -


Good to see you again
